Vamos a extraer el organismo de la entidad adjudicadora. Primera prueba: /boe/dias/2012/03/08/pdfs/BOE-B-2012-7789.pdf

In [51]:
from tools.download import download_pdf
import os

download_pdf("/boe/dias/2012/03/08/pdfs/BOE-B-2012-7789.pdf")

PDF BOE-B-2012-7789.pdf downloaded successfully.


He creado un modulo data_downloader para dividir texto en pedazos. Pero he hecho la prueba y con una resolucion de una pagina extrae bien la informacion, asi que para este caso no lo uso. Seguro que para un futuro viene bien.

In [52]:
from tools.data_loader import PDFReader
pdf_file = "BOE-B-2012-7789.pdf"
pdf_path = os.path.join("pdfs", pdf_file)
# First, create an instance of PDFReader with the path to your PDF.
pdf_reader = PDFReader(pdf_path=pdf_path, chunk_size=500, overlap=30)

# Then, retrieve a specific part (for example, the first page) of the document.
# Replace 1 with the page number you want to retrieve.
document_part = pdf_reader

print(document_part)


Part(BOLETÍN OFICIAL DEL ESTADO
Núm. 58 Jueves 8 de marzo de 2012 Sec. V-A.  Pág. 10420
cve: BOE-B-2012-7789
 V. Anuncios
 A. Anuncios de licitaciones públicas y adjudicaciones
 MINISTERIO DE FOMENTO
7789 Resolución de la Autoridad Portuaria de Barcelona por la que se
efectúa anuncio previo de la licitación "Asistencia Técnica para la
coordinación en seguridad y salud en fase de proyecto y ejecución de
las diferentes obras del puerto de Barcelona y prevención en riesgos
: {'page': 1})
Part(elona y prevención en riesgos
laborales".
1. Entidad adjudicadora: Datos generales y datos para la obtención de la
información:
a) Organismo: Ministerio de Fomento - Autoridad Portuaria de Barcelona.
b) Dependencia que tramita el expediente: Secretaría General.
c) Domicilio: Edificio Este del WTCB.
d) Localidad y código postal: Barcelona 08039.
e) Teléfono: 93 298 60 00.
f) Telefax: 93 298 60 01.
g) Correo electrónico: sau@portdebarcelona.es.
h) Dirección de Interne: {'page': 1})
Part(na.es.
h) Direc

In [53]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import instructor

Extraigo entidad adjudicadora. Resultado debe ser: *Ministerio de Fomento - Autoridad Portuaria de Barcelona*

In [54]:
class Entidad_adjudicadora(BaseModel):
    organismo: str = Field(..., title="Organismo", description="Nombre de la entidad adjudicadora")

In [55]:
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",
    ),
    mode=instructor.Mode.JSON,
)


In [56]:
import PyPDF2

with open("pdfs/BOE-B-2012-7789.pdf", "rb") as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()

In [57]:
resp = client.chat.completions.create(
    model="llama3",
    messages=[
        {
            "role": "user",
            #"content": content.format(texto=result)
            "content": text
        }
    ],
    response_model=Entidad_adjudicadora,
    max_retries=10
)

In [58]:
resp

Entidad_adjudicadora(organismo='Ministerio de Fomento - Autoridad Portuaria de Barcelona')

Exito

Extraemos la cantidad de dinero del contrato

La mejor forma de filtrar por convocatoria/contratacion es por el siguiente subtitulo que tiene el pdf Y NO LOS XML:
- A. Anuncios de licitaciones públicas y adjudicaciones
- A. Contratación del Sector Público

Asi que hecho un mini filtro con `boe/licitaciones.ipynb` y ahora hago la extraccion de la empresa etc de una contratacion

In [59]:
from pydantic import BaseModel, Field
class Cantidad_contrato(BaseModel):
    cantidad: float = Field(..., title="Cantidad", description="Cantidad del contrato. Siempre la cantidad total del contrato")
    divisa: str = Field(..., title="Divisa", description="Divisa en la que se ha realizado el contrato")

In [60]:
from tools.data_loader import PDFReader

pdf_file = "licitaciones_filtered_pdfs/licitaciones/licitaciones_filtered_pdfs/contratacion/BOE-B-2015-17810.pdf"

pdf = PDFReader(pdf_path=pdf_file, chunk_size=500, overlap=30)

text = pdf.get_part(7)

print(text)

Part(Verificable en http://www.boe.es
d) Criterios de adjudicación: Descripción de la actividad (máximo 12 puntos),
elaboración de informes (máximo 12 puntos), organización y medios
propuestos (máximo 4 puntos) y plan de calidad propuesto (máximo 12
puntos).
4. Valor estimado del contrato: 611.570,24 euros.
5. Presupuesto base de licitación:
a) Importe neto: 305.785,12 euros. Importe total: 370.000,00 euros.
6. Garantías exigidas. Definitiva (%): 5.
7. Requisitos específ: {'page': 2})


In [61]:
import instructor
from openai import OpenAI
content = f"""
Te voy a mostrar el texto que he extraído del PDF y luego me vas a decir la cantidad del contrato. En el PDF se pueden referir a la cantidad como 'Valor estimado del contrato', 'valor aproximado del contrato', etc. Aquí tienes el texto:
{text}
"""
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",
    ),
    mode=instructor.Mode.JSON,
)
resp = client.chat.completions.create(
    model="llama3",
    messages=[
        {
            "role": "user",
            #"content": content.format(texto=result)
            "content": content
        }
    ],
    response_model=Cantidad_contrato,
    max_retries=10
)

In [63]:
resp.cantidad

61157024.0

Probamos el nuevo modulo

In [1]:
from tools.data_loader import PDFReader

pdf_file = "licitaciones_filtered_pdfs/licitaciones/licitaciones_filtered_pdfs/contratacion/BOE-B-2015-17810.pdf"

pdf = PDFReader(pdf_path=pdf_file, chunk_size=500, overlap=30)

text_amount = pdf.get_part(7)

print(text_amount)

Part(Verificable en http://www.boe.es
d) Criterios de adjudicación: Descripción de la actividad (máximo 12 puntos),
elaboración de informes (máximo 12 puntos), organización y medios
propuestos (máximo 4 puntos) y plan de calidad propuesto (máximo 12
puntos).
4. Valor estimado del contrato: 611.570,24 euros.
5. Presupuesto base de licitación:
a) Importe neto: 305.785,12 euros. Importe total: 370.000,00 euros.
6. Garantías exigidas. Definitiva (%): 5.
7. Requisitos específ: {'page': 2})


In [2]:
from tools.data_extractors import DataExtractor

extractor_amount = DataExtractor(text=text_amount)

extractor_amount.extract_amount()

None
